I will be performing a survival analysis of the amount of days on Ibruntinib, where the event of interest is death (everything else is censored). 

I'll be applying the cox proportional hazards model. 

The relevant independent variables are: 
age of diagnosis, gender, platelet count, anti-coagulation, antiplatelet, invasive procedure (which I believe is PMHx bleeding risk), (maybe INR later), Molecular/cytogenetics, anemia, prior lines of therapy, major bleed (major vs. minor vs. none)

The relevant output variables are: duration on ibrunitinb and Death (by last FU) 

Load necessary libraries 

In [1]:
library("survival")
library("survminer")

print("complete")

Loading required package: ggplot2
Loading required package: ggpubr


[1] "complete"


Load dataset

In [3]:
# Import the data and look at the first six rows
bleed_data <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Mina/Bleeding_study/Ibrutinib Data Set, July 13,2021, de- identified data_cleaned_forSurvAnal.csv')
## bleed_data_cyto is the dataset with the rows that have 'unknown' cytogenetics removed. Only use this data set 
## when plotting survival curves where cytogenetics are the stratification factor 
bleed_data_cyto <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Mina/Bleeding_study/Ibrutinib Data Set, July 13,2021, de- identified data_cleaned_forSurvAnal_CtyoCleaned.csv')
head(bleed_data_cyto)

X,Age.at.diagnosis,gender,Platelets...50..Y.N.,Plt.Nadir.while.on.Ibrutinib,Anemia..hb...100...Y.N.,Anemia..hb...110...Y.N.,HR.Molecular.Cytogenetics..Y.N.,Prior.lines.of.therapy,Prior.lines.of.therapy.1,...,PMHx.bleeding.risk..Y.N.,Major.Bleed..Y.N.,Major.Bleed,Minor.Bleed,duration.of.ibrutinib..in.days.,Death..only.by.last.FU.,Bleed.1_MajOrMin.1...censored..2...event.,Bleed.1_Time_on_Ibruntinib,Bleed.2_MajOnly.1...censored..2...event.,Bleed.2_Time_on_Ibruntinib
0,48,0,1,15,1,1,1,1,1,...,1,0,0,0,670,1,1,670,1,670
1,66,1,0,93,1,1,1,0,0,...,0,1,0,1,502,1,2,466,1,502
2,74,0,0,189,1,1,1,1,1,...,0,0,0,0,103,1,1,103,1,103
3,53,0,1,40,0,0,0,2,0,...,0,0,0,0,1316,2,1,1316,1,1316
4,60,1,1,49,0,1,0,3,0,...,0,2,1,0,2086,2,2,NA,2,NA
5,51,1,1,16,1,1,0,2,0,...,0,0,0,0,1870,1,1,1870,1,1870


Univariate Cox regression analysis

In [26]:
res.cox <- coxph(Surv(Bleed.2_Time_on_Ibruntinib,Bleed.2_MajOnly.1...censored..2...event.) ~ anti.platelet..Y.N., data =  bleed_data)
summary(res.cox)

Call:
coxph(formula = Surv(Bleed.2_Time_on_Ibruntinib, Bleed.2_MajOnly.1...censored..2...event.) ~ 
    anti.platelet..Y.N., data = bleed_data)

  n= 168, number of events= 16 
   (1 observation deleted due to missingness)

                      coef exp(coef) se(coef)     z Pr(>|z|)
anti.platelet..Y.N. 0.4190    1.5204   0.5779 0.725    0.468

                    exp(coef) exp(-coef) lower .95 upper .95
anti.platelet..Y.N.      1.52     0.6577    0.4898     4.719

Concordance= 0.549  (se = 0.058 )
Likelihood ratio test= 0.49  on 1 df,   p=0.5
Wald test            = 0.53  on 1 df,   p=0.5
Score (logrank) test = 0.53  on 1 df,   p=0.5


Univariate 

Multivariate Cox regression analysis

In [27]:
# excluding HR molecular cytogenetics because then I would have to 
# eliminate all the rows with "unknown" cytogenetics 

res.cox <- coxph(Surv(Bleed.2_Time_on_Ibruntinib,Bleed.2_MajOnly.1...censored..2...event.) ~  Plt.Nadir.while.on.Ibrutinib + anticoagulation..Y.N. + Anemia..hb...100...Y.N., data = bleed_data)
summary(res.cox)

Call:
coxph(formula = Surv(Bleed.2_Time_on_Ibruntinib, Bleed.2_MajOnly.1...censored..2...event.) ~ 
    Plt.Nadir.while.on.Ibrutinib + anticoagulation..Y.N. + Anemia..hb...100...Y.N., 
    data = bleed_data)

  n= 168, number of events= 16 
   (1 observation deleted due to missingness)

                                  coef exp(coef)  se(coef)      z Pr(>|z|)  
Plt.Nadir.while.on.Ibrutinib -0.005031  0.994982  0.005606 -0.897   0.3696  
anticoagulation..Y.N.         1.257831  3.517783  0.524479  2.398   0.0165 *
Anemia..hb...100...Y.N.       1.378314  3.968204  0.608248  2.266   0.0234 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

                             exp(coef) exp(-coef) lower .95 upper .95
Plt.Nadir.while.on.Ibrutinib     0.995     1.0050    0.9841     1.006
anticoagulation..Y.N.            3.518     0.2843    1.2584     9.833
Anemia..hb...100...Y.N.          3.968     0.2520    1.2046    13.072

Concordance= 0.773  (se = 0.061 )
Likelihood ratio test

Plot survival curves if I want? http://www.sthda.com/english/wiki/cox-proportional-hazards-model It explains how to do so here